In [1]:
import os

In [2]:
%pwd

'c:\\Users\\JOY\\Desktop\\DL_Project\\Renel_Disease_Classification_Deep_Learning\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\JOY\\Desktop\\DL_Project\\Renel_Disease_Classification_Deep_Learning'

In [5]:
import tensorflow as tf

In [6]:
model = tf.keras.models.load_model('artifacts/training/model.h5')

In [27]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class EvaluationConfig:
    path_of_model: Path
    training_data:Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [31]:
from Renel_Disease_Classifier.constants import *
from Renel_Disease_Classifier.utils.common import read_yaml,create_directories,save_json

class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifact_root])


    def get_evaluation_config(self)-> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts\data_ingestion\kidney_new\CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone",
            mlflow_uri="https://dagshub.com/jatin9091/Renel_Disease_Classification_Deep_Learning.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )

        return eval_config



In [32]:
import mlflow
import tensorflow as tf
from pathlib import Path
import mlflow.keras
from urllib.parse import  urlparse
import dagshub
from mlflow.models.signature import infer_signature

class Evaluation:
    def __init__(self,config:EvaluationConfig):
        self.config = config

    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation='bilinear',
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(

            **datagenerator_kwargs

        )

        self.valid_generator = valid_datagenerator.flow_from_directory(

            directory=self.config.training_data,

            subset="validation",
            shuffle= False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(path=Path("artifacts/training/model.h5"))
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)
        self.save_score()


    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        dagshub.init(repo_owner='jatin9091', repo_name='Renel_Disease_Classification_Deep_Learning', mlflow=True)

        # Get a batch of input data from the validation generator
        # Ensure that `self.valid_generator` has been initialized
        if not hasattr(self, 'valid_generator'):
            self._valid_generator()

        # Fetch one batch of data
        input_data, _ = next(self.valid_generator)
        
        # Generate predictions
        predictions = self.model.predict(input_data)

        # Infer the model signature
        signature = infer_signature(input_data, predictions)

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]})
            
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(
                    self.model, "model",signature=signature, registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model",signature=signature)
         


In [33]:
try:
    config = ConfigurationManager()
    val_config = config.get_evaluation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e

Found 178 images belonging to 2 classes.
9/9 [==============================] - 32s 4s/step - loss: 5.0394 - accuracy: 0.6180


Initialized MLflow to track repo "jatin9091/Renel_Disease_Classification_Deep_Learning"

Repository jatin9091/Renel_Disease_Classification_Deep_Learning initialized!

1/1 [==============================] - 4s 4s/step
INFO:tensorflow:Assets written to: C:\Users\JOY\AppData\Local\Temp\tmp3bj7do0f\model\data\model\assets


Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/09/29 03:37:30 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 3
Created version '3' of model 'VGG16Model'.
